In [90]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib as plot
import math
import talib
from sklearn import metrics, linear_model, preprocessing, svm, model_selection
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
conn = sqlite3.connect('poloniex_0.1.db')
c = conn.cursor()

In [108]:
df = pd.read_sql_query("SELECT * FROM candles_USDT_ETH", conn)
df = df.drop('id' ,1)
df = df.drop('trades' ,1)
df['start'] = pd.to_datetime(df['start'],unit='s')
df.set_index('start' , inplace = True)
#df.index.dtype
df.head(10)
#print (len(df.index))


,open,high,low,close,vwp,volume
start,,,,,,
2017-06-12 14:37:00,376.500010,376.500010,375.423819,376.50000,375.984977,6.203722
2017-06-12 14:38:00,376.435997,376.480000,375.000000,376.40001,376.234584,13.728674
2017-06-12 14:39:00,375.000000,376.000000,374.105980,376.00000,375.278772,11.562601
2017-06-12 14:40:00,375.657617,376.000000,374.105980,376.00000,375.938314,6.251341
2017-06-12 14:41:00,376.400000,376.600000,376.000100,376.60000,376.448518,63.735705
2017-06-12 14:42:00,376.000100,376.600000,376.000000,376.00000,376.200211,6.293728
2017-06-12 14:43:00,376.600000,376.600000,375.000000,375.00000,375.729590,147.057915
2017-06-12 14:44:00,376.600000,376.909634,376.000000,376.01014,376.259007,143.724666
2017-06-12 14:45:00,376.010140,377.962200,376.010140,377.96220,376.243165,9.771476


In [109]:
def safediv(x,y):
    if y == 0 :
        return np.nan
    else:
        return x/y

In [110]:
def ohlcsum(df):
    #df = df.sort()
    
    return {
       'open': df['open'][0],
       'high': df['high'].max(),
       'low': df['low'].min(),
       'close': df['close'][-1],
       'volume': df['volume'].sum(),
        
        'vwp': safediv((df['vwp']* df['volume']).sum(),df['volume'].sum()) 
        
      }

In [111]:
def convertcandles(df, mins):
    s = str(mins) + 'T'
    return( df.resample(s).agg(ohlcsum))
    
    
#convertcandles(df, 120)
    

In [112]:
newdf = convertcandles(df,15)

#newdf = df
price = np.array(newdf['vwp'])
h = np.array(newdf['high'])
l = np.array(newdf['low'])
c = np.array(newdf['close'])
#sma  = talib.SMA(price,50)
m = talib.MACD(price, 12,26,9)

mom = talib.MOM(price,12 )
rsi = talib.RSI(price)
krsi, drsi = talib.STOCHRSI(price)
roc = talib.ROC(price)
mean5d = pd.rolling_mean(newdf['vwp'],5)
mean10d = pd.rolling_mean(newdf['vwp'],10)
newdf['willR'] = talib.WILLR(h,l,c)
newdf['krsi'] = krsi
newdf['drsi'] = drsi
newdf['disp5d'] = newdf['vwp'] - mean5d
newdf['disp10d'] = newdf['vwp'] - mean10d
newdf['cci'] = talib.CCI(h,l,c)
newdf['roc'] = roc
#m[1][40:60]
# newdf['msig'] = np.where((m[1]>m[2]),1,0)
newdf['msig'] = m[1]-m[2]
#newdf['mom'] = np.where(mom>0,1,0)
# newdf['rsi'] = np.where((rsi>50)& (rsi> np.roll(rsi,1)),1,0)
newdf['rsi'] = rsi
# newdf['sma'] = np.where((pd.rolling_mean(newdf['vwp'],9)- pd.rolling_mean(newdf['vwp'],26))>0,1,0)
newdf['target'] = np.where((newdf['close'].shift(1)- newdf['close'])>0,1,0)
newdf.dropna(axis = 0, how = 'any', inplace = True)
newdf.drop(newdf.index[0], inplace = True)
newdf.head(30)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
  from ipykernel import kernelapp as app
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=10,center=False).mean()
  app.launch_new_instance()


,open,high,low,close,vwp,volume,willR,krsi,drsi,disp5d,disp10d,cci,roc,msig,rsi,target
start,,,,,,,,,,,,,,,,
2017-06-12 23:00:00,365.000000,369.010000,363.180070,366.600000,366.661512,700.902526,-15.641956,100.000000,5.143878e+01,6.175700,5.807458,75.820657,4.264295,-5.790336,55.313702,0
2017-06-12 23:15:00,369.010000,372.100000,366.698870,368.605090,369.946330,856.692872,-12.278061,100.000000,8.477211e+01,7.993254,6.837489,91.679620,6.490493,-4.843681,57.043865,0
2017-06-12 23:30:00,368.605090,375.899930,368.000000,375.780000,372.281407,596.078005,-0.286230,100.000000,1.000000e+02,6.976365,7.704899,123.589423,4.104160,-3.876449,58.280458,0
2017-06-12 23:45:00,375.780000,387.000000,375.000000,386.500070,381.250669,1849.340033,-1.063681,100.000000,1.000000e+02,10.681201,15.016772,192.889563,4.544816,-3.218468,62.719813,0
2017-06-13 00:00:00,386.500070,388.424670,374.000000,381.000000,380.966464,2035.884340,-15.992941,96.936060,9.897869e+01,6.745188,13.379639,148.827059,3.682067,-2.195476,62.492895,1
2017-06-13 00:15:00,381.999930,383.927000,375.700000,383.927000,380.946402,981.872276,-11.126053,95.699356,9.754514e+01,3.868148,12.164369,124.562430,3.239096,-1.009502,62.475711,0
2017-06-13 00:30:00,383.000000,385.000000,381.085000,383.700064,383.333978,1445.963658,-11.687433,100.000000,9.754514e+01,3.578194,12.479548,118.577441,5.715222,0.103199,63.753069,1
2017-06-13 00:45:00,383.700064,385.500000,383.577574,385.500000,384.557825,921.776825,-7.234864,100.000000,9.856645e+01,2.346757,10.799770,107.272252,8.167226,1.205200,64.421630,0
2017-06-13 01:00:00,384.000290,389.979720,384.000290,389.978790,387.197423,444.850183,-0.002215,100.000000,1.000000e+02,3.797005,10.212480,107.573546,9.091656,2.191743,65.883243,0


In [113]:
factors = ['msig','rsi','cci','willR','krsi','drsi','disp5d','disp10d']
X = newdf[factors]
Y = newdf['target']

In [114]:
Xtrain,  Xtest, Ytrain, Ytest = model_selection.train_test_split(X,Y)
#Xtrain.head()
scaler = preprocessing.MinMaxScaler(copy = False)
Xtrain =scaler.fit_transform(Xtrain)
mod = linear_model.LogisticRegression(solver = "saga")
mod.fit(Xtrain, Ytrain)
Xtest =scaler.transform(Xtest)
print('LogisticRegression score: %f'
      % mod.score(Xtest, Ytest))

LogisticRegression score: 0.719457


In [115]:
# names = [ "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes"]
names = [ "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost"
         ]
# classifiers = [SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1),
#     AdaBoostClassifier(),
#     GaussianNB()]
classifiers = [SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier()]
for name, clf in zip(names, classifiers):
    clf.fit(Xtrain, Ytrain)
    score = clf.score(Xtest, Ytest)
    print(name, score)

('Linear SVM', 0.67873303167420818)
('RBF SVM', 0.74660633484162897)
('Decision Tree', 0.72398190045248867)
('Random Forest', 0.69004524886877827)
('Neural Net', 0.70135746606334837)
('AdaBoost', 0.75113122171945701)
